In [4]:
using LightGraphs
using Random
using Shuffle
using Distributions
using StatsBase
# using Plots
using LinearAlgebra
using SparseArrays
using Arpack
using ArgParse
using Distributed
# using TickTock
using DataFrames
using CSV
using DelimitedFiles


# ==============================
# =  GLOBAL MODEL PARAMETERS   =
# ==============================

# --- Network size ---
num_agents = 100                      # N: number of nodes/agents

# --- Physical-contact layer (Watts–Strogatz small-world) ---
sw_avg_degree = 6                     # k: expected degree in small-world
sw_rewire_prob = 0.3                  # betaS: rewiring probability in WS model

# --- Social layer (Klimek–Thurner friendship-like evolution) ---
kt_triadic_closure_prob = 0.58        # r: probability of triadic closure
kt_turnover_prob        = 0.12        # p: node turnover probability
kt_new_links_on_rejoin  = 1           # m: # new links when a node re-enters

# Bias to reuse an existing neighbor from the physical layer when adding new edges
# (encodes overlap between physical and social contacts)
overlap_bias_prob = 1.0              


# ==========================
# =  EPIDEMIC DYNAMICS     =
# ==========================

# NOTE: Here β is the continuous-time transmission rate (λ).
# In the event-driven simulation we sample Exponential with 1/β etc.
transmission_rate_λ = 6.0             # beta
recovery_rate        = 1.0            # mu

num_sir_runs_per_season = 10 * num_agents  # Nsim: # Monte Carlo SIR runs per season
num_seasons               = 1000            # vacCycles: # of decision seasons


# ==========================
# =  LEARNING PAYOFFS      =
# ==========================

cost_infection   = 1.0               # CI: material cost from getting infected
cost_vaccination = 0.1               # CV: material cost from vaccination

memory_len = 4                       # mem: length of memory window (seasons)

memChoice_len = memory_len           
memPayoff_len = memory_len           


# ==========================
# =  SOCIAL NORMS/WEIGHTS  =
# ==========================

# Exogenous pro-vaccination baseline (e.g., public health recommendation)
injunctive_baseline = 0.4            # GVac

# Base learning-rate for norms (scales the three per-channel rates below)
base_norm_lr = 1.0                   # LearnRate

# Per-channel learning-rate multipliers (personal y, injunctive ŷ, descriptive x̃)
Xi_y_lr     = 0.01                  # XiY
Xi_yT_lr    = 0.1                   # XiYTilded
Xi_xT_lr    = 1.0                   # XiXTilded

# Actual per-channel learning rates
lr_y      = Xi_y_lr  * base_norm_lr # YLearnRate
lr_yT     = Xi_yT_lr * base_norm_lr # YTidedLearnRate
lr_xT     = Xi_xT_lr * base_norm_lr # XTildedLearnRate

# Convex combination weights when blending own vs. social/authority inputs
gamma_y   = 1.0                      # gammaY
gamma_yT  = 1.0                      # gammaYTilded
gamma_xT  = 1.0                      # gammaXTilded

# Information observability and uncertainty
obs_fraction = 1.0                   # fraction of neighbors observed
min_uncertainty_floor = 0.1          # minimum uncertainty lower bound

# Regret (Loomes–Sugden) parameters for material payoffs
regret_coeff = 0.0                   # η₁-like coefficient
regret_power = 0.0                   # η₂-like curvature

# Quantal-response / logit rationality parameter
rationality_k = 0.1                  # krat

# Window size for reporting final averages
stats_window = 400                   # windowSN

# RNG seed
rng_seed = abs(rand(Int, 1)[1])
Random.seed!(rng_seed)


# ==========================
# =  EVENT-DRIVEN SIR      =
# ==========================
# Epidemic states used by this code:
#   1 = Susceptible (S)
#   2 = Infected (I)
#   3 = Recovered after being Infected (R from I)
#   4 = Recovered from a “blocked” infection attempt because vaccinated (R from V)
#
# Vaccination states used by this code:
#   1 = Unvaccinated
#   2 = Vaccinated
#
# NOTE: Those integers are preserved to keep structure intact.

function event_driven(recovery_rate::Float64,
                      λ_transmission::Float64,
                      contact_neighbors::Vector{Any},          # neighbors on physical layer
                      vacc_state::Vector{Int},                 # 1=unvaccinated, 2=vaccinated
                      epi_state_sim::Vector{Int})              # initial S/I states for this SIR run
    # Exponential clocks for recovery and infection
    global dmu   = Exponential(1 / recovery_rate)
    global dlamb = Exponential(1 / λ_transmission)

    # Priority-queue as a sorted vector of (time, [src, dst]) “events”
    # dst==src encodes a recovery event for node src
    global event_queue = Vector{Any}(undef, 0)

    # Seed events: for each initially infected node schedule a recovery time, and infection attempts to susceptible neighbors
    for u in 1:num_agents
        if epi_state_sim[u] == 2                 # infected
            global rec_time = rand(dmu)
            push!(event_queue, [rec_time, [u, u]])   # recovery event for u

            # schedule infection attempts on susceptible, unvaccinated neighbors
            for v in 1:length(contact_neighbors[u])
                nbr = contact_neighbors[u][v]
                if epi_state_sim[nbr] == 1 && vacc_state[nbr] == 1
                    global inf_time = rand(dlamb)
                    if inf_time < rec_time         # only if infection occurs before u recovers
                        # Replace earlier later-in-time attempt on same target with earlier one
                        global idx = 1
                        global replaced = 0
                        while idx <= length(event_queue) && replaced == 0
                            if event_queue[idx][2][2] == nbr
                                if inf_time < event_queue[idx][1]
                                    event_queue[idx] = deepcopy([inf_time, [u, nbr]])
                                end
                                replaced = 1
                            end
                            idx += 1
                        end
                        if replaced == 0
                            push!(event_queue, [inf_time, [u, nbr]])
                        end
                    end
                end
            end
        end
        # keep queue time-sorted
        event_queue = sort(event_queue, by = x -> x[1])
    end

    # Process the queue until empty
    global empty_flag = 0
    while empty_flag == 0
        if event_queue[1][2][1] == event_queue[1][2][2]
            # Recovery event
            epi_state_sim[event_queue[1][2][1]] = 3
        elseif epi_state_sim[event_queue[1][2][2]] == 1 && event_queue[1][2][1] != event_queue[1][2][2] && vacc_state[event_queue[1][2][2]] == 1
            # Successful infection event onto susceptible & unvaccinated target
            global new_inf = deepcopy(event_queue[1][2][2])
            global epi_state_sim[new_inf] = 2
            global rec_time = rand(dmu) + event_queue[1][1]
            push!(event_queue, [rec_time, [new_inf, new_inf]])   # schedule recovery of new_inf

            # Schedule infection attempts from newly infected to its neighbors
            for k in 1:length(contact_neighbors[new_inf])
                nbr = contact_neighbors[new_inf][k]
                if epi_state_sim[nbr] == 1 && vacc_state[nbr] == 1
                    global inf_time = rand(dlamb) + event_queue[1][1]
                    if inf_time < rec_time
                        global idx = 1
                        global replaced = 0
                        while idx <= length(event_queue) && replaced == 0
                            if event_queue[idx][2][2] == nbr
                                if inf_time < event_queue[idx][1]
                                    event_queue[idx] = deepcopy([inf_time, [new_inf, nbr]])
                                end
                                replaced = 1
                            end
                            idx += 1
                        end
                        if replaced == 0
                            push!(event_queue, [inf_time, [new_inf, nbr]])
                        end
                    end
                end
            end
        elseif epi_state_sim[event_queue[1][2][2]] == 1 && event_queue[1][2][1] != event_queue[1][2][2] && vacc_state[event_queue[1][2][2]] == 2
            # Attempted infection on vaccinated → mark as 4 (blocked)
            epi_state_sim[event_queue[1][2][2]] = 4
        end

        deleteat!(event_queue, 1)
        if length(event_queue) > 0
            event_queue = deepcopy(sort(event_queue, by = x -> x[1]))
        else
            empty_flag = 1
        end
    end
    return epi_state_sim
end


# ==========================
# =  INITIALIZATIONS       =
# ==========================

# --- payoff memory (per agent) ---
payoff_history_init = Vector{Any}(undef, num_agents)
for i in 1:num_agents
    tmp = Vector{Float64}(undef, 0)
    for _ in 1:memPayoff_len
        push!(tmp, -rand())  # start with negative payoffs so first updates are informative
    end
    payoff_history_init[i] = tmp
end
global payoff_history = deepcopy(payoff_history_init)


# --- Network generators (layer 1: physical; layer 2: social) ---

# Small-world physical-contact network
function small_world_network(n::Int, k::Int, rewire_prob::Float64)
    return watts_strogatz(n, k, rewire_prob)
end

# Klimek–Thurner social network, biased to overlap with the physical layer G1
function klimek_thurner_network(n::Int,
                                triadic_p::Float64,
                                turnover_p::Float64,
                                new_links::Int,
                                physical_G::SimpleGraph,
                                overlap_bias_prob::Float64)
    g = SimpleGraph(n)

    # Initialize: each node adds a link—preferentially to an existing physical neighbor with prob overlap_bias_prob
    for i in 1:n
        if rand() < overlap_bias_prob
            potential = Random.shuffle(neighbors(physical_G, i))   # assume connected enough
            count = 0
            s = potential[count + 1]
            add_edge!(g, i, s)
        else
            s = rand(1:n)
            add_edge!(g, i, s)
        end
    end

    # Evolve edges to mimic closure + turnover
    for _ in 1:(5n)
        i = rand(1:n)

        if degree(g, i) < 2
            # If degree small, attach to a physical neighbor not yet connected in g
            cand = setdiff(neighbors(physical_G, i), neighbors(g, i))
            if rand() < overlap_bias_prob && !isempty(cand)
                potential = Random.shuffle(cand)
                count = 1
                s = potential[count]
                while has_edge(g, i, s) == true && count < length(potential)
                    count += 1
                    s = potential[count]
                end
                add_edge!(g, i, s)
            else
                s = rand(1:n)
                while has_edge(g, i, s) == true
                    s = rand(1:n)
                end
                add_edge!(g, i, s)
            end

        else
            # Pick a neighbor j of i and either close triad or rewire via physical layer
            neighbors_i  = neighbors(g, i)
            j = rand(neighbors_i)

            neighbors_iG1 = neighbors(physical_G, i)
            inter         = intersect(neighbors_i, neighbors_iG1)

            potential_tri_close = Random.shuffle(setdiff(inter, [j]))
            potential_any       = Random.shuffle(setdiff(neighbors_i, [j]))

            if rand() < triadic_p
                # Triadic closure (prefer overlap with G1 if overlap_bias_prob is high)
                if rand() < overlap_bias_prob && !isempty(potential_tri_close)
                    count = 1
                    s = potential_tri_close[count]
                    while has_edge(g, j, s) == true && count < length(potential_tri_close)
                        count += 1
                        s = potential_tri_close[count]
                    end
                    add_edge!(g, j, s)
                else
                    count = 1
                    s = potential_any[count]
                    while has_edge(g, j, s) == true && count < length(potential_any)
                        count += 1
                        s = potential_any[count]
                    end
                    add_edge!(g, j, s)
                end
            else
                # Rewire guided by physical layer structure
                potential = Random.shuffle(setdiff(neighbors(physical_G, j), neighbors(g, j)))
                if rand() < overlap_bias_prob && !isempty(potential)
                    count = 1
                    s = potential[count]
                    while has_edge(g, j, s) == true && count < length(potential)
                        count += 1
                        s = potential[count]
                    end
                    add_edge!(g, j, s)
                else
                    s = rand(1:n)
                    while has_edge(g, j, s) == true
                        s = rand(1:n)
                    end
                    add_edge!(g, j, s)
                end
            end
        end

        # Turnover: wipe edges of a random node and reconnect it
        if rand() < turnover_p
            node_to_reset = rand(1:n)
            for nb in neighbors(g, node_to_reset)
                rem_edge!(g, node_to_reset, nb)
            end
            for _ in 1:new_links
                cand_p = setdiff(neighbors(physical_G, node_to_reset), neighbors(g, node_to_reset))
                potential = Random.shuffle(cand_p)
                if rand() < overlap_bias_prob && !isempty(potential)
                    count = 1
                    s = potential[count]
                    while has_edge(g, node_to_reset, s) == true && count < length(potential)
                        count += 1
                        s = potential[count]
                    end
                    add_edge!(g, node_to_reset, s)
                else
                    s = rand(1:n)
                    while has_edge(g, node_to_reset, s) == true
                        s = rand(1:n)
                    end
                    add_edge!(g, node_to_reset, s)
                end
            end
        end
    end
    return g
end

# Build layers
physical_layer = small_world_network(num_agents, sw_avg_degree, sw_rewire_prob)
social_layer   = klimek_thurner_network(num_agents, kt_triadic_closure_prob, kt_turnover_prob,
                                        kt_new_links_on_rejoin, physical_layer, overlap_bias_prob)

# Adjacency lists
neighbors_phys = Vector{Any}(undef, 0)
for i in 1:num_agents
    push!(neighbors_phys, deepcopy(neighbors(physical_layer, i)))
end

neighbors_social = Vector{Any}(undef, 0)
for i in 1:num_agents
    push!(neighbors_social, deepcopy(neighbors(social_layer, i)))
end


# ==========================
# =  INITIAL STATES        =
# ==========================

# Vaccination states (1=unvaccinated, 2=vaccinated) — initially all 1
vacc_state_init = Vector{Int}(undef, num_agents)
for i in 1:num_agents
    vacc_state_init[i] = 1
end
vacc_state_init = Random.shuffle(vacc_state_init)
global vacc_state = deepcopy(vacc_state_init)

# Choice memory (history of vaccination choices per agent, integers in {1,2})
choice_history_init = Vector{Any}(undef, num_agents)
for i in 1:num_agents
    v = Vector{Int}(undef, 0)
    for _ in 1:memChoice_len
        push!(v, rand(1:2))
    end
    choice_history_init[i] = deepcopy(v)
end
global choice_history = deepcopy(choice_history_init)

# Social-norm variables per agent (all in [0,1])
#   Y        : personal norm y (preferred behavior)
#   YTilded       : perceived injunctive norm ŷ (peers' moral approval)
#   pref_xT      : perceived descriptive norm x̃ (peers' intentions)
global Y   = rand(num_agents)
global YTilded  = rand(num_agents)
global XTilded = rand(num_agents)


# ==========================
# =  NORM METRICS          =
# ==========================

# phi_Vac: “stability/consistency” score in neighbors’ recent choices (closer to 1 = stable/consistent)
# typeF=1 → compute from neighbors on social layer; typeF=2 → compute from agent’s own history only
function phi_Vac(N::Int, neigh2::Vector{Any}, memL::Int, choice_hist::Vector{Any}, typeF::Int64)
    phi_vec = ones(N)
    for i in 1:N
        count_len = (typeF == 1) ? length(neigh2[i]) : 1
        if count_len > 0
            hmat = zeros(Float64, count_len, 2)  # history frequency of (Unvac,Vac)
            rmat = zeros(Float64, count_len, 2)  # most recent action as one-hot

            if typeF == 1
                # From neighbors
                for jidx in 1:length(neigh2[i])
                    j = neigh2[i][jidx]
                    h = [0.0, 0.0]
                    for jj in 1:memL
                        if choice_hist[j][jj] == 1
                            h .+= [1/memL, 0.0]
                        elseif choice_hist[j][jj] == 2
                            h .+= [0.0, 1/memL]
                        end
                    end
                    hmat[jidx, :] = h
                    # recency vector
                    rmat[jidx, :] = (choice_hist[j][1] == 1) ? [1.0, 0.0] : [0.0, 1.0]
                end
            else
                # From own history only
                h = [0.0, 0.0]
                for jj in 1:memL
                    if choice_hist[i][jj] == 1
                        h .+= [1/memL, 0.0]
                    elseif choice_hist[i][jj] == 2
                        h .+= [0.0, 1/memL]
                    end
                end
                hmat[1, :] = h
                rmat[1, :] = (choice_hist[i][1] == 1) ? [1.0, 0.0] : [0.0, 1.0]
            end

            # Mean squared mismatch between history and recency over both actions
            dif1 = hmat[:, 1] .- rmat[:, 1]
            dif2 = hmat[:, 2] .- rmat[:, 2]
            s1 = 0.0; s2 = 0.0
            for t in 1:length(hmat[:, 1])
                s1 += dif1[t]^2
                s2 += dif2[t]^2
            end
            Sit = (s1 + s2) / length(neigh2[i])  # instability proxy
            phi_vec[i] = 1 - Sit/2               # map to [0,1]
        end
    end
    return phi_vec
end

# X′: consensus magnitude around the agent: 2*| (vaccinated_fraction) - 0.5 |
function X_prime(N::Int, vaccinated_neighbor_counts::Vector{Float64}, neigh2::Vector{Any})
    Xp = ones(N)
    for i in 1:N
        Xp[i] = 2 * abs((vaccinated_neighbor_counts[i] / length(neigh2[i])) - 0.5)
    end
    return Xp
end

# Geometric mean utility
function geom_mean(vec::Vector{Float64})
    prod = 1.0
    for v in vec
        prod *= v
    end
    return prod^(1 / length(vec))
end

# Uncertainty- & fear-weighted trust into social info (both SD and SI are the same here by design)
function trust_weights(N::Int,
                       Xprime::Vector{Float64},
                       phiVac::Vector{Float64},
                       safety_vec::Vector{Float64})

    trust_S = 0.5 .* ones(N)   # trust for injunctive sources

    for i in 1:N
        
        fear        = 1 - safety_vec[i]

        # weight of norms grows when there is consensus (X′), stability (φ), fear, and a minimum uncertainty
        w = geom_mean([phiVac[i], Xprime[i], fear, min_uncertainty_floor])
        trust_S[i] = w
    end
    return trust_S
end

# Precompute initial φ from neighbors and from own history
phi_neighbors = phi_Vac(num_agents, neighbors_social, memChoice_len, choice_history, 1)
phi_own       = phi_Vac(num_agents, neighbors_social, memChoice_len, choice_history, 2)


# ==========================
# =  ONE SEASON SIM        =
# ==========================

# Compute E[P] and Var[P] for infection with partially observed neighbors
function infection_probability_stats_from_lambda(n_total::Int, n_obs::Int, k_obs::Int,
                                                 λ::Float64, p_infected::Float64)
    # Convert rate to per-step infection probability
    dt = 1.0
    β_prob = 1 - exp(-λ * dt)

    n_unobs = n_total - n_obs
    mean_p  = 0.0
    mean_p2 = 0.0

    for x in 0:n_unobs
        # Binomial prob. that x of the unobserved neighbors are infected
        prob_x = binomial(n_unobs, x) * p_infected^x * (1 - p_infected)^(n_unobs - x)
        k_total = k_obs + x

        # Infection probability given k_total infected neighbors
        p_inf = 1 - (1 - β_prob)^k_total

        mean_p  += prob_x * p_inf
        mean_p2 += prob_x * p_inf^2
    end

    var_p = mean_p2 - mean_p^2
    return mean_p, var_p
end

# Small-thresholding helper
function threshold_small(x, k=10)
    abs(x) < 10.0^-k ? 0.0 : x
end

# Early-stopping check: last n values within relative ±limit of their mean
function check_approx_equal(vec, n, limit)
    if length(vec) < n
        return false
    end
    last_vals = vec[end-n+1:end]
    return all(abs.(last_vals .- mean(last_vals)) .<= limit * mean(last_vals))
end

# Update (y, ŷ, x̃) after a decision round
function update_norms!(XTilded_vec::Vector{Float64},     # x̃
                       Y_vec::Vector{Float64},        # y
                       YTilded_vec::Vector{Float64},       # ŷ
                       lr_y::Float64, lr_yT::Float64, lr_xT::Float64,
                       phi_neighbors::Vector{Float64},
                       prefVac::Vector{Float64},
                       gamma_y::Float64, gamma_yT::Float64, gamma_xT::Float64,
                       injunctive_baseline::Float64,
                       neigh2::Vector{Any},
                       vaccinated_neighbor_counts::Vector{Float64},
                       trust::Vector{Float64},
                       safety_vec::Vector{Float64})

    Y_prev  = deepcopy(Y_vec)
    YTilded_prev = deepcopy(YTilded_vec)

    for i in 1:num_agents
        # Perceived “pressure to vaccinate” from neighbors’ current adoption
        # (fraction of neighbors NOT vaccinated → pressure to vaccinate)
        pref_press_from_peers = (length(neigh2[i]) - vaccinated_neighbor_counts[i]) / length(neigh2[i])

        # Blend own vs social using trust weights 
        t = trust[i]

        # Personal norm y: move towards (own prefVac vs peer pressure), with small anchor on injunctive_baseline
        Y_vec[i] += lr_y * ( gamma_y * ((1 - t)*prefVac[i] + t*pref_press_from_peers) + (1 - gamma_y)*injunctive_baseline - Y_vec[i] )
        Y_vec[i]  = min(max(Y_vec[i], 0.0), 1.0)

        # Injunctive norm ŷ: similar blending rule
        YTilded_vec[i] += lr_yT * ( gamma_yT * ((1 - t)*Y_prev[i] + t*pref_press_from_peers) + (1 - gamma_yT)*injunctive_baseline - YTilded_vec[i] )
        YTilded_vec[i]  = min(max(YTilded_vec[i], 0.0), 1.0)

        # Descriptive norm x̃: track neighbors’ apparent intention/pressure
        XTilded_vec[i] += lr_xT * ( gamma_xT * ((1 - t)*YTilded_prev[i] + t*pref_press_from_peers) + (1 - gamma_xT)*injunctive_baseline - XTilded_vec[i] )
        XTilded_vec[i]  = min(max(XTilded_vec[i], 0.0), 1.0)
    end
    return Y_vec, YTilded_vec, XTilded_vec
end

# Run many SIR realizations for a season; update payoff history; return outbreak stats and perceived risks
function run_one_season!(N::Int, Nsim::Int, vacc_state::Vector{Int},
                         recovery_rate::Float64, λ::Float64,
                         neighbors_phys::Vector{Any},
                         payoff_historyS::Vector{Any}, memPayoff_len::Int)

    epi_total_sizes   = Vector{Any}(undef, 0)   # final sizes per SIR run
    epi_state_per_run = Vector{Any}(undef, 0)   # final states S/I/R for each run

    sim = 0
    while sim < Nsim
        # Seed initial infected set size 
        global epi_init = Vector{Int}(undef, N)
        global epi_tmp  = Vector{Int}(undef, N)

        # Number of initially infected equals max( min(N - #vaccinated, 1), 1 ) = 1 unless everyone vaccinated
        global n_init_inf = maximum([minimum([Int(round((N - count(x -> (x == 2), vacc_state)))), 1]), 1])
        for u in 1:n_init_inf
            epi_tmp[u] = 2
        end
        for u in (n_init_inf + 1):N
            epi_tmp[u] = 1
        end
        epi_tmp = Random.shuffle(epi_tmp)

        # Ensure no vaccinated individuals start infected; if so, swap in a susceptible unvaccinated replacement
        for u in 1:N
            if (epi_tmp[u] == 2) && (vacc_state[u] == 2)
                touched = Int[]
                swapped = 0
                while swapped == 0 && length(touched) < N
                    r = rand(1:N)
                    if r in touched
                        # continue
                    else
                        if epi_tmp[r] == 1 && vacc_state[r] == 1
                            epi_tmp[u], epi_tmp[r] = epi_tmp[r], epi_tmp[u]
                            swapped = 1
                        else
                            push!(touched, r)
                        end
                    end
                end
            end
        end

        global epi_for_run = deepcopy(epi_tmp)
        global epi_final   = event_driven(recovery_rate, λ, neighbors_phys, vacc_state, epi_for_run)

        # outbreak size = # ever infected or blocked (state in {3,4})
        push!(epi_total_sizes, N - (count(s -> (s == 1), epi_final) + count(s -> (s == 4), epi_final)))
        push!(epi_state_per_run, epi_final)

        sim += 1
    end

    # Compute per-agent realized infection rate and neighbors’ infection rate across runs
    infected_rate     = zeros(1, N)
    infected_nb_rate  = zeros(1, N)
    for i in 1:N
        self_sum = 0.0
        nb_sum   = 0.0
        for r in 1:length(epi_state_per_run)
            state_r = epi_state_per_run[r]
            if state_r[i] == 3 || state_r[i] == 4
                self_sum += 1
            end
            # Average infected neighbors in this run (only direct infections count)
            nb_infected_frac = 0.0
            for nb in neighbors_phys[i]
                if state_r[nb] == 3
                    nb_infected_frac += 1 / length(neighbors_phys[i])
                end
            end
            nb_sum += nb_infected_frac
        end
        infected_rate[1, i]    = self_sum / Nsim
        infected_nb_rate[1, i] = nb_sum / Nsim
    end

    outbreak_size = mean(epi_total_sizes) / N

    # Perceived susceptibility E[P] and uncertainty Var[P] (with partial observability)
    unc_suscept = zeros(N)
    mean_suscept = zeros(N)

    for i in 1:N
        payoff = 0.0

        n_total = length(neighbors_phys[i])
        n_obs   = round(Int, obs_fraction * n_total)
        k_obs   = round(Int, infected_nb_rate[1, i] * n_obs)

        mP, vP = infection_probability_stats_from_lambda(n_total, n_obs, k_obs, λ, outbreak_size)

        unc_suscept[i]  = threshold_small(vP)^0.5
        mean_suscept[i] = mP

        # End-of-season “experienced” payoff sample 
        if vacc_state[i] == 1
            payoff += -cost_infection * (infected_rate[1, i])        # unvaccinated → realized infection rate
        else
            payoff += -cost_infection * (mean_suscept[i])            # vaccinated → expected susceptibility
        end

        ph = deepcopy(payoff_historyS[i])
        pushfirst!(ph, payoff)
        payoff_historyS[i] = deepcopy(getindex(ph, 1:memPayoff_len))
    end

    return outbreak_size, infected_rate, infected_nb_rate, payoff_historyS, unc_suscept, mean_suscept
end


# ==========================
# =  TRAJECTORY STORAGE    =
# ==========================

global infected_frac_traj = Float64[]   # ⟨I⟩ across seasons
global vaccinated_frac_traj = Float64[] # ⟨V⟩ across seasons

global pref_intention_traj = Float64[]  # ⟨prefVac⟩
global xT_traj            = Float64[]   # ⟨x̃⟩
global y_traj             = Float64[]   # ⟨y⟩
global yT_traj            = Float64[]   # ⟨ŷ⟩


# Count vaccinated neighbors (with optional self weight)
global vaccinated_neighbor_counts = zeros(num_agents)
for i in 1:num_agents
    countV = 0.0

    for nb in neighbors_social[i]
        if vacc_state[nb] == 2
            countV += 1
        end
    end
    vaccinated_neighbor_counts[i] = countV
end

global Xprime = X_prime(num_agents, vaccinated_neighbor_counts, neighbors_social)


# ==========================
# =  REGRET ON PAYOFFS     =
# ==========================

# Apply Loomes–Sugden regret/rejoice on (π_V, π_¬V)
function regret_payoffs(u_V::Float64, u_notV::Float64; regret_coeff::Float64, regret_power::Float64)
    regret_V    = max(0.0, u_notV - u_V)
    regret_notV = max(0.0, u_V - u_notV)

    πV    = u_V    - regret_coeff * regret_V^regret_power
    π_not = u_notV - regret_coeff * regret_notV^regret_power
    return πV, π_not
end


# ==========================
# =  MAIN DYNAMICS LOOP    =
# ==========================

global t_season = 1
global stop_flag = 0

while t_season < num_seasons && stop_flag == 0
    # Push the latest choice into memory (front = most recent)
    for i in 1:num_agents
        ch = deepcopy(choice_history[i])
        pushfirst!(ch, vacc_state[i])
        choice_history[i] = deepcopy(getindex(ch, 1:memChoice_len))
    end

    if count(x -> x == 2, vacc_state) == num_agents + 1

        global stop_flag = 1
        push!(vaccinated_frac_traj, 1.0)
    else
        push!(vaccinated_frac_traj, count(x -> x == 2, vacc_state) / num_agents)

        global outbreak_size, infected_rate, infected_nb_rate, payoff_history,
               unc_suscept, mean_suscept =
            run_one_season!(num_agents, num_sir_runs_per_season, vacc_state,
                            recovery_rate, transmission_rate_λ, neighbors_phys,
                            payoff_history, memPayoff_len)

        push!(infected_frac_traj, outbreak_size)

        # Compute per-agent intention to vaccinate from payoffs + norms
        global prefVac = zeros(num_agents)  # “intention weight” in [0,1]
        global safety_vec = zeros(num_agents)

        for i in 1:num_agents
            if length(neighbors_social[i]) > 0
                # Safety proxy (higher = safer)
                if vacc_state[i] == 2
                    safety_vec[i] = min(1 - mean_suscept[i], 1)
                else
                    safety_vec[i] = min(1 - infected_rate[1, i], 1)
                end

                # Build discounted average payoffs from memory 
                sumV = 0.0; wV = 0.0
                sumU = 0.0; wU = 0.0
                for jj in 1:memChoice_len
                    impactV = 1.0
                    impactU = 1.0
                    if choice_history[i][jj] == 2
                        sumV += (-cost_vaccination) * safety_vec[i]^(jj - 1)
                        wV   += safety_vec[i]^(jj - 1)
                        sumU += impactU * payoff_history[i][jj] * safety_vec[i]^(jj - 1)
                        wU   += impactU * safety_vec[i]^(jj - 1)
                    else
                        sumV += impactV * (-cost_vaccination) * safety_vec[i]^(jj - 1)
                        wV   += impactV * safety_vec[i]^(jj - 1)
                        sumU += payoff_history[i][jj] * safety_vec[i]^(jj - 1)
                        wU   += safety_vec[i]^(jj - 1)
                    end
                end

                avPay_V   = (wV == 0) ? 0.0 : 1.0 + (sumV / wV)
                avPay_not = (wU == 0) ? 0.0 : 1.0 + (sumU / wU)

                # Uncertainty & fear 
                global max_unc = 0.25
                global uncertainty = clamp(min_uncertainty_floor + unc_suscept[i] / max_unc, 0.0, 1.0)
                global fear        = 1 - safety_vec[i]

                # Apply regret on material payoffs → p_learn
                πV_reg, πU_reg = regret_payoffs(avPay_V, avPay_not; regret_coeff=regret_coeff, regret_power=regret_power)
                p_learn = 0.5 + 0.5 * (πU_reg - πV_reg)     

                # Inertia (own recency stability)
                inertia = geom_mean([phi_own[i], 1.0 - fear, uncertainty])

                # Weights among channels 
                phi_Emp = geom_mean([fear, uncertainty])                          # individual vs descriptive
                phi_Col = geom_mean([phi_neighbors[i], Xprime[i], fear, uncertainty]) # descriptive vs learning
                theta_Col = geom_mean([phi_neighbors[i], Xprime[i], fear, min_uncertainty_floor]) # individual vs perceived

                # Final “intention” proxy in [0,1] (naming preserved)
                prefVac[i] = minimum([1.0, maximum([0.0,
                   (phi_Emp) * ((1 - phi_Col) * p_learn + (phi_Col) * XTilded[i]) +
                   (1 - phi_Emp) * ((1 - theta_Col) * ((1 - inertia) * p_learn + inertia * Y[i]) +
                                    (theta_Col) * YTilded[i])
                ])])

                # Map intention into a logit probability.
                # IMPORTANT: As in your original code, the assignment below inverts the meaning:
                # if rand() < p then state ← 1 (unvaccinated); else state ← 2 (vaccinated).
                # I’m keeping this *exactly* as-is to preserve structure and results.
                p = 1 / (1 + exp(-(2 * prefVac[i] - 1) / rationality_k))

                old_state = vacc_state[i]
                if rand() < p
                    vacc_state[i] = 1       # Unvaccinated if probability is high
                else
                    vacc_state[i] = 2       # Vaccinated otherwise
                end

                # Update vaccinated-neighbor counts if state changed
                diff = vacc_state[i] - old_state
                if diff != 0
                    for nb in neighbors_social[i]
                        vaccinated_neighbor_counts[nb] += diff
                    end
                end
            end
        end

        # Refresh consensus & trust inputs for next season
        global Xprime         = X_prime(num_agents, vaccinated_neighbor_counts, neighbors_social)
        global phi_neighbors  = phi_Vac(num_agents, neighbors_social, memChoice_len, choice_history, 1)
        global trust = trust_weights(num_agents, Xprime, phi_neighbors, safety_vec)
        global phi_own        = phi_Vac(num_agents, neighbors_social, memChoice_len, choice_history, 2)

        # Update norms (y, ŷ, x̃)
        global Y, YTilded, XTilded =
            update_norms!(XTilded, Y, YTilded, lr_y, lr_yT, lr_xT,
                          phi_neighbors, prefVac, gamma_y, gamma_yT, gamma_xT, injunctive_baseline,
                          neighbors_social, vaccinated_neighbor_counts, trust, safety_vec)

        # Store aggregates
        push!(pref_intention_traj, mean(prefVac))
        push!(xT_traj,            mean(XTilded))
        push!(y_traj,             mean(Y))
        push!(yT_traj,            mean(YTilded))
    end

    # Early stop when both infected and vaccinated fractions stabilize
    if check_approx_equal(infected_frac_traj, 10, 0.05) &&
       check_approx_equal(vaccinated_frac_traj, 10, 0.05)
        stop_flag = 1
    end

    global t_season += 1
end


# ==========================
# =  LOGGING / OUTPUT      =
# ==========================

# Pack parameters into a vector (names made consistent with the variables above)
Params = [
    num_agents,
    transmission_rate_λ,
    lr_y,
    lr_yT,
    cost_infection,
    memory_len,
    cost_vaccination,
    num_sir_runs_per_season,
    num_seasons,
    injunctive_baseline,
    # gamma aggregate not present; include the three separately for reproducibility:
    gamma_y,
    gamma_yT,
    gamma_xT,
    obs_fraction,
    min_uncertainty_floor,
    rationality_k,
    regret_coeff,
    regret_power,
    rng_seed
]

# Simple numeric hash-like fold
global fold_sum
fold_sum = 0.0
for i in 1:length(Params)-1
    fold_sum += Params[i] * 10^(i - 3.0)
end

run_id = 0
if run_id == 0
    params_fname = string("Params_", fold_sum, "_", run_id, "_Holme.csv")
    writedlm(params_fname, Params, ',')
end

# Rolling-window means for reporting
info_matrix = [
    mean(infected_frac_traj[length(infected_frac_traj)-stats_window:length(infected_frac_traj)]),
    mean(vaccinated_frac_traj[length(vaccinated_frac_traj)-stats_window:length(vaccinated_frac_traj)])
]

info_fname = string("InfoMatrix_", fold_sum, "_", run_id, ".csv")
writedlm(info_fname, info_matrix, ',')


In [5]:
println(info_matrix)

[0.04618765586034913, 0.7849625935162097]
